## Ensure a Minikube cluster is running

- If you created a Minikube cluster with the command `minikube start --nodes 3` and it is still running:
  - Do nothing
- If you have a stopped (not deleted) Minikube cluster that was created with the command `minikube start --nodes 3`:
  - Run the command `minikube start`
- If you don't have a Minikube cluster (or deleted your previous one):
  - Run the command `minikube start --nodes 3`  

## Enable the Ingress addon

In [1]:
!minikube addons enable ingress

💡  ingress is an addon maintained by Kubernetes. For any concerns contact minikube on GitHub.
You can view the list of minikube maintainers at: https://github.com/kubernetes/minikube/blob/master/OWNERS
    ▪ Using image registry.k8s.io/ingress-nginx/kube-webhook-certgen:v20231011-8b53cabe0
    ▪ Using image registry.k8s.io/ingress-nginx/kube-webhook-certgen:v20231011-8b53cabe0
    ▪ Using image registry.k8s.io/ingress-nginx/controller:v1.9.4
🔎  Verifying ingress addon...
🌟  The 'ingress' addon is enabled


## Deploy two Delopyments

- The first deployment:
  - Is called `web` and has `replicas` set to 1.
  - Has a Pod template with a container:
    - Listening on `containerPort` 8080
    - Based on the image `gcr.io/google-samples/hello-app:1.0`.
  - The image hosts a web server with a default web page that returns:
    - `Hello, World!`
    - `Version: 1.0.0`
- The second deployment:
  - Is called `web2` and has `replicas` set to 1.
  - Has a Pod template with a container:
    - Listening on `containerPort` 8080
    - Based on the image `gcr.io/google-samples/hello-app:2.0`.
  - The image hosts a web server with a default web page that returns:
    - `Hello, World!`
    - `Version: 2.0.0`
- Both images are pulled from Google's container registry `gcr.io`.

In [2]:
!kubectl create deployment web --image=gcr.io/google-samples/hello-app:1.0
!kubectl create deployment web2 --image=gcr.io/google-samples/hello-app:2.0

deployment.apps/web created
deployment.apps/web2 created


## List Deployments

- We see both Deployments running.
  - The `web` Deployment is using image `gcr.io/google-samples/hello-app:1.0`.
  - The `web2` Deployment is using image `gcr.io/google-samples/hello-app:2.0`.

In [3]:
#!kubectl get deploy -o wide
!kubectl get deployments -o wide

NAME   READY   UP-TO-DATE   AVAILABLE   AGE   CONTAINERS   IMAGES                                SELECTOR
web    1/1     1            1           10s   hello-app    gcr.io/google-samples/hello-app:1.0   app=web
web2   1/1     1            1           10s   hello-app    gcr.io/google-samples/hello-app:2.0   app=web2


## Get Pods

- We see that two Pods are running.

In [4]:
#!kubectl get po -o wide
!kubectl get pods -o wide

NAME                    READY   STATUS    RESTARTS   AGE   IP            NODE           NOMINATED NODE   READINESS GATES
web-57f46db77f-w8gmc    1/1     Running   0          33s   10.244.2.9    minikube-m03   <none>           <none>
web2-866dc4bcc8-lfdt2   1/1     Running   0          33s   10.244.1.11   minikube-m02   <none>           <none>


## Deploy a ClusterIP Service for each Deployment

- Each Service:
  - Is of type `ClusterIP`.
  - Listens on `port` 8080.
  - Redirects traffic to `targetPort` 8080.

In [5]:
!kubectl expose deployment web --type ClusterIP --port 8080 --target-port 8080
!kubectl expose deployment web2 --type ClusterIP --port 8080 --target-port 8080

service/web exposed
service/web2 exposed


## List Services

- We see the two `ClusterIP` Services for `web` and `web2`.

In [6]:
#!kubectl get svc -o wide
!kubectl get services -o wide

NAME         TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)    AGE   SELECTOR
kubernetes   ClusterIP   10.96.0.1       <none>        443/TCP    11h   <none>
web          ClusterIP   10.99.20.12     <none>        8080/TCP   5s    app=web
web2         ClusterIP   10.105.160.27   <none>        8080/TCP   5s    app=web2


## Deploy an Ingress

- The Ingress' definition is in the YAML file `manifests/ingress.yaml`.

In [7]:
!kubectl apply -f manifests/ingress.yaml

ingress.networking.k8s.io/example-ingress created


## Let's look at the Ingress' YAML

```bash
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: example-ingress                               # the ingress' name
  annotations:
    nginx.ingress.kubernetes.io/rewrite-target: /$1   # this rewrites the target so that it is prefixed with a slash /
spec:
  rules:
  - host: hello-world.info   # the host determines which domain the ingress triggers on ("Host" in the request header)
    http:
      paths:

      - path: /                # this path is taken if the URL has the default path as a prefix
        pathType: Prefix
        backend:
          service:
            name: web          # the name of the ClusterIP service to redirect to  (the "web" service in this case)
            port:
              number: 8080     # the port to redirect the traffic to (8080)
      
      - path: /v2                   # this path is taken if the URL has the path /v2 as a prefix
        pathType: Prefix
        backend:
          service:
            name: web2              # the name of the ClusterIP service to redirect to  (the "web2" service in this case)
            port:
              number: 8080          # the port to redirect the traffic to (8080)
```

In [8]:
#!type manifests/ingress.yaml #use this on Windows
!cat manifests/ingress.yaml

apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: example-ingress
  annotations:
    nginx.ingress.kubernetes.io/rewrite-target: /$1
spec:
  rules:
  - host: hello-world.info
    http:
      paths:
      - path: /
        pathType: Prefix
        backend:
          service:
            name: web
            port:
              number: 8080
      - path: /v2
        pathType: Prefix
        backend:
          service:
            name: web2
            port:
              number: 8080

## List Ingresses

- We see that the Ingress triggers on
  - Host `hello-world.info` (`HOSTS`).
  - Port 80 (`PORTS`).

In [9]:
!kubectl get ingress

NAME              CLASS   HOSTS              ADDRESS   PORTS   AGE
example-ingress   nginx   hello-world.info             80      13s


## Access the first Application via the Ingress

- The Ingres redirects the traffic to the first application if:
  - The `Host` header is `hello-world.info`.
  - The `path` is the deault path (`/`).
- The first application is based on the image `gcr.io/google-samples/hello-app:1.0`.

In [10]:
#!curl --resolve "hello-world.info:80:$( minikube ip )" -i http://hello-world.info
!curl -H "Host: hello-world.info" "http://$( minikube ip)"

Hello, world!
Version: 1.0.0
Hostname: web-57f46db77f-w8gmc


## Access the second Application via the Ingress

- The Ingres redirects the traffic to the econdsecond application if:
  - The `Host` header is `hello-world.info`.
  - The `path` is (`/v2`).
- The first application is based on the image `gcr.io/google-samples/hello-app:2.0`.

In [11]:
#!curl --resolve "hello-world.info:80:$( minikube ip )" -i http://hello-world.info/v2
!curl -H "Host: hello-world.info" "http://$( minikube ip)/v2"

Hello, world!
Version: 2.0.0
Hostname: web2-866dc4bcc8-lfdt2


## Delete the Ingress, Services and Deployments

In [12]:
!kubectl delete -f manifests/ingress.yaml
!kubectl delete service web
!kubectl delete service web2
!kubectl delete deployment web
!kubectl delete deployment web2

ingress.networking.k8s.io "example-ingress" deleted
service "web" deleted
service "web2" deleted
deployment.apps "web" deleted
deployment.apps "web2" deleted


## List Ingresses, Services, Deployments and Pods

- We see that the Ingress, ClusterIP Services and Deployments with associated Pods have been deleted.

In [13]:
!kubectl get ingress
!kubectl get services
!kubectl get deployments
!kubectl get pods

No resources found in default namespace.
NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   11h
No resources found in default namespace.
No resources found in default namespace.
